In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [5]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-07-25/"
files_start = 2336
files_end = 2337


In [6]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [7]:
filenames = os.listdir(path_all)

In [8]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [9]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2016-07-25 21:39:00,1,blank sol,0,0,0.000747,0.000556,0.004614,0.001063,0.002888,0.000914,...,0.000057,1.068630e-05,0.000766,2.462390e-05,0.000577,5.347130e-05,0.004597,0.001090,0.002939,0.000905
2016-07-25 21:39:00,2,blank sol,0,0,0.000713,0.000522,0.004351,0.000998,0.002720,0.000869,...,0.000065,3.140160e-05,0.000814,3.683490e-05,0.000611,3.714590e-05,0.004993,0.001171,0.003196,0.001004
2016-07-25 21:39:00,3,blank sol,0,0,0.000987,0.000711,0.005759,0.001327,0.003708,0.001184,...,0.000026,2.567560e-05,0.000854,-5.388000e-07,0.000622,3.987840e-05,0.004995,0.001160,0.003210,0.001022
2016-07-25 21:39:00,4,blank sol,0,0,0.000707,0.000538,0.004351,0.000951,0.002708,0.000828,...,0.000074,1.403400e-06,0.000661,3.081100e-06,0.000468,1.866270e-05,0.004051,0.000895,0.002534,0.000758
2016-07-25 21:39:00,5,blank sol,0,0,0.000643,0.000453,0.003995,0.000889,0.002453,0.000804,...,0.000019,6.406300e-06,0.000750,4.904560e-05,0.000549,1.228700e-05,0.004494,0.001028,0.002784,0.000872
2016-07-25 21:39:00,6,blank sol,0,0,0.000713,0.000508,0.004355,0.000982,0.002690,0.000883,...,0.000024,9.260200e-06,0.000739,3.801600e-06,0.000558,2.222330e-05,0.004426,0.001041,0.002796,0.000865
2016-07-25 21:39:00,7,blank sol,0,0,0.000810,0.000618,0.004858,0.001104,0.003093,0.001004,...,0.000042,1.711210e-05,0.000805,1.025250e-05,0.000575,1.508410e-05,0.004673,0.001093,0.002965,0.000942
2016-07-25 21:39:00,8,blank sol,0,0,0.000826,0.000622,0.004930,0.001102,0.003102,0.000971,...,0.000052,1.425530e-05,0.000730,1.455930e-05,0.000565,4.348440e-05,0.004547,0.001041,0.002851,0.000906
2016-07-25 21:39:00,9,blank sol,0,0,0.000654,0.000447,0.003976,0.000885,0.002481,0.000796,...,0.000034,1.069130e-05,0.000667,2.103750e-05,0.000464,1.369790e-05,0.004026,0.000941,0.002541,0.000775


In [10]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2016-07-25 19:27:00,0,0,0.000751,0.000572,0.004596,0.001050,0.002852,0.000911,0.002342,0.000055,...,0.000017,0.000706,2.221715e-05,0.000528,0.000034,0.004271,0.000954,0.002602,0.000827,blank sol
2016-07-25 19:56:00,0,0,0.000776,0.000577,0.004713,0.001065,0.002917,0.000938,0.002372,0.000059,...,0.000001,0.000833,7.745950e-07,0.000615,0.000038,0.004898,0.001118,0.003070,0.000970,blank sol
2016-07-25 20:20:00,0,0,0.000863,0.000637,0.005226,0.001197,0.003300,0.001053,0.002529,0.000052,...,0.000005,0.000911,6.785805e-06,0.000680,0.000020,0.005391,0.001243,0.003443,0.001077,blank sol
2016-07-25 21:39:00,0,0,0.000685,0.000503,0.004185,0.000933,0.002599,0.000834,0.002123,0.000039,...,0.000016,0.000722,1.690563e-05,0.000527,0.000031,0.004359,0.001002,0.002740,0.000857,blank sol
2016-07-25 22:58:00,0,0,0.000793,0.000589,0.004735,0.001092,0.003027,0.000964,0.002214,0.000058,...,0.000016,0.000758,5.918265e-06,0.000564,0.000017,0.004557,0.001049,0.002903,0.000923,blank sol
2016-07-26 00:17:00,0,0,0.000829,0.000623,0.004937,0.001158,0.003149,0.001018,0.002370,0.000067,...,-0.000004,0.000827,1.217376e-05,0.000626,0.000017,0.004886,0.001126,0.003105,0.000981,blank sol
2016-07-26 01:35:00,0,0,0.004244,0.003288,0.023332,0.005925,0.016493,0.005188,0.010207,0.000314,...,0.000015,0.001512,1.070656e-04,0.001151,0.000151,0.008550,0.002080,0.005792,0.001826,blank sol
2016-07-26 02:54:00,0,0,0.000880,0.000666,0.005233,0.001232,0.003399,0.001079,0.002447,0.000062,...,0.000003,0.000801,1.040190e-05,0.000608,0.000026,0.004799,0.001104,0.003085,0.000983,blank sol
2016-07-26 04:13:00,0,0,0.000897,0.000680,0.005354,0.001258,0.003498,0.001113,0.002472,0.000053,...,0.000011,0.001014,1.086593e-05,0.000767,0.000010,0.005920,0.001405,0.003898,0.001227,blank sol


In [11]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [13]:
df_all.to_csv(path_all + "blanks.csv", mode='w', header=True)

In [215]:
df_internal_sub.to_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm.csv", mode='a', header=False)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")